In [21]:
from langchain import PromptTemplate, LLMChain
from langchain_openai import AzureOpenAI
#from langchain.llms import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
LANGCHAIN_TRACING_V2=os.environ["LANGCHAIN_TRACING_V2"] 
LANGCHAIN_PROJECT=os.environ["LANGCHAIN_PROJECT"]
LANGCHAIN_ENDPOINT=os.environ["LANGCHAIN_ENDPOINT"]
LANGCHAIN_API_KEY=os.environ["LANGCHAIN_API_KEY"]

In [23]:
from langsmith import Client
client = Client()

### Intro to LangChain

LangChain is a popular framework that allows users to quickly build apps and pipelines around Large Language Models. It can be used for chatbots, Generative Question-Answering (GQA), summarization, and much more.

The core idea of the library is that we can "chain" together different components to create more advanced use-cases around LLMs. Chains may consist of multiple components from several modules:

- **Prompt templates**: Prompt templates are, well, templates for different types of prompts. Like "chatbot" style templates, ELI5 question-answering, etc.

- **LLMs**: Large language models like GPT-3, BLOOM, etc.

- **Agents**: Agents use LLMs to decide what actions should be taken. Tools like web search or calculators can be used, and all packaged into a logical loop of operations.

- **Memory**: Short-term memory, long-term memory.


### Prompt Template

In [24]:
template = """Question: {question}

Answer: """
prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

# user question
question = "Which NFL team won the Super Bowl in the 2010 season?"

In [25]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])



qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

### Using LLM in LangChain-AzureOpenAI


In [26]:
davinci = AzureOpenAI(
    deployment_name="text-davinci-003", 
    model_name="text-davinci-003",
    api_version = os.environ.get("AZURE_OPENAI_DEPLOYMENT_VERSION"),
    api_key = os.environ.get("AZURE_OPENAI_KEY")
)

In [27]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

print(llm_chain.invoke(question))

{'question': 'Which NFL team won the Super Bowl in the 2010 season?', 'text': ' The Green Bay Packers won the Super Bowl in the 2010 season.'}


In [28]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
llm_chain.generate(qs)

LLMResult(generations=[[Generation(text=' The Green Bay Packers won the Super Bowl in the 2010 season.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' 192.4 centimeters', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' Edgar Mitchell was the 12th person to walk on the moon. He was part of the Apollo 14 mission in 1971.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' A blade of grass does not have any eyes.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 126, 'prompt_tokens': 75, 'completion_tokens': 51}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('c9745ebe-6ecc-4162-a2a3-d83b1a49a073')), RunInfo(run_id=UUID('e3da7fd8-ec07-4fc9-87b3-9c9415a684cb')), RunInfo(run_id=UUID('8bd886fc-b040-4222-85d4-2f9e26886d17')), RunInfo(run_id=UUID('66e59051-aa46-4e91-8256-8fc54ab8f1a5'))])

In [29]:
llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

The New Orleans Saints won the Super Bowl in the 2010 season.
If you are 6 ft 4 inches, you are 193.04 centimeters tall.
The 12th person on the moon was Harrison Schmitt.
A blade of grass has no eyes.


In [30]:
qs = [
    "Which NFL team won the Super Bowl in the 2010 season?",
    "If I am 6 ft 4 inches, how tall am I in centimeters?",
    "Who was the 12th person on the moon?",
    "How many eyes does a blade of grass have?"
]
print(llm_chain.run(qs))

1. The New Orleans Saints won the Super Bowl in the 2010 season.
2. 6 ft 4 inches is 193.04 centimeters.
3. The 12th person on the moon was Harrison Schmitt.
4. A blade of grass does not have eyes.
